<a href="https://colab.research.google.com/github/Dean-Clark27/CSE337_RoboPianist/blob/main/Dean_Clark_CSE337_Robopianist_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<b>Robo Pianist (Learning Expressive Piano Playing via Reinforcement Learning)</b>

### Final Project by: Dean Clark CSE 337



In [1]:
# @title Run to install robopianist
from IPython.display import clear_output
import subprocess

if subprocess.run("nvidia-smi").returncode:
    raise RuntimeError(
        "Cannot communicate with GPU. "
        "Make sure you are using a GPU Colab runtime. "
        "Go to the Runtime menu and select Choose runtime type."
    )

# Install dependencies.
%shell bash <(curl -s https://raw.githubusercontent.com/google-research/robopianist/main/scripts/install_deps.sh) --no-soundfonts --no-menagerie

print("Installing robopianist...")
%pip install -q robopianist>=1.0.6

%env MUJOCO_GL=egl

clear_output()
%shell echo Installed $(robopianist --version)


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):
Installed robopianist 1.0.10


In [ ]:
!pip install stable-baselines3
!pip install shimmy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 kB 5.2 MB/s eta 0:00:00


In [ ]:
# @title Necessary Imports for the Project
from IPython.display import HTML
from base64 import b64encode
import numpy as np
from robopianist.suite.tasks import self_actuated_piano
from robopianist.suite.tasks import piano_with_shadow_hands
from dm_env_wrappers import CanonicalSpecWrapper
from robopianist.wrappers import PianoSoundVideoWrapper
from robopianist import music
from mujoco_utils import composer_utils
import dm_env
import gym
from gym import spaces

/usr/local/lib/python3.12/dist-packages/pydub/utils.py:14: DeprecationWarning: 'audioop' is deprecated and slated for removal in Python 3.13
  import audioop
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware ob

In [ ]:
# @title Helper functions


# Reference: https://stackoverflow.com/a/60986234.
def play_video(filename: str):
    mp4 = open(filename, "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    return HTML(
        """
  <video controls>
        <source src="%s" type="video/mp4">
  </video>
  """
        % data_url
    )


def inspect_shadow_hand_actuators(env):
    """
    Print all actuator names to identify finger vs. wrist controls
    """
    physics = env.physics

    print("Total actuators:", physics.model.nu)
    print("\nActuator names:")
    for i in range(physics.model.nu):
        name = physics.model.id2name(i, 'actuator')
        print(f"  {i}: {name}")

In [ ]:
import numpy as np
import dm_env
import gym
from gym import spaces
from dm_env import TimeStep, StepType # Explicitly import if needed for StepType.LAST comparison, or ts.last() handles it.

class DMEnvToGym(gym.Env):
    """
    Convert a dm_env environment (like RoboPianist) to Gymnasium API.
    Correctly handles (obs, reward, discount, step_type).
    """

    metadata = {"render_modes": []}

    def __init__(self, dm_env):
        super().__init__()
        self._env = dm_env

        # Observation space:
        # Assuming dm_env.observation_spec() already provides a simple spec for the flattened observation
        # from CustomRoboWrapper which returns `{"obs": specs.Array(shape=(self.obs_dim,), dtype=np.float32)}`
        obs_spec = dm_env.observation_spec()["obs"]
        self.observation_space = spaces.Box(
            low=float(obs_spec.minimum) if hasattr(obs_spec, 'minimum') else -np.inf,
            high=float(obs_spec.maximum) if hasattr(obs_spec, 'maximum') else np.inf,
            shape=obs_spec.shape,
            dtype=obs_spec.dtype
        )

        # Action space from dm_env
        act_spec = dm_env.action_spec()
        self.action_space = spaces.Box(
            low=act_spec.minimum,
            high=act_spec.maximum,
            dtype=act_spec.dtype, # Use act_spec.dtype
            shape=act_spec.shape,
        )

    def _flatten(self, obs):
        # Case 1: Already flat array (e.g., {"obs": array} after CustomRoboWrapper)
        if isinstance(obs, np.ndarray):
            return obs.astype(np.float32)

        # Case 2: Dict of fields → flatten into one array (if a different wrapper precedes CustomRoboWrapper)
        if isinstance(obs, dict):
            return np.concatenate([v.ravel() for v in obs.values()]).astype(np.float32)

        raise TypeError(f"Unknown observation type: {type(obs)}")


    def reset(self, *, seed=None, options=None):
        ts = self._env.reset()
        obs = self._flatten(ts.observation)
        info = {} # Gymnasium reset returns (observation, info)
        return obs, info

    def step(self, action):
        ts = self._env.step(action)
        obs = self._flatten(ts.observation)
        reward = ts.reward if ts.reward is not None else 0.0
        terminated = ts.last() # This correctly checks if it's the last step
        truncated = False # Assuming no explicit truncation signal for now.
        info = {} # Gymnasium step returns (observation, reward, terminated, truncated, info)
        return obs, reward, terminated, truncated, info

In [ ]:
class RoboPianistWrapper(gym.Env):
    """
    Wraps the RoboPianist / Shadow Hand MuJoCo environment to:
      - expose a 10-dim observation (as in your proposal)
      - expose a continuous action vector (e.g., joint position deltas / torques)
      - compute the reward Rt = w_hit*rhit + w_timing*rtiming + w_energy*renergy + w_smooth*rsmooth
    Replace the calls to `self.env` methods to match the real env.
    """
    def __init__(self,
                 base_env,
                 action_dim=45,
                 obs_dim=10,
                 dt=0.02,
                 hit_threshold=0.5,
                 sigma_timing=0.05,
                 reward_weights=None):
        super().__init__()
        self.env = base_env
        self.dt = dt
        self.hit_threshold = hit_threshold
        self.sigma_timing = sigma_timing

        if reward_weights is None:
            reward_weights = dict(w_hit=1.0, w_timing=1.0, w_energy=-0.01, w_smooth=-0.01)
        self.rw = reward_weights

        # Action: continuous vector (tweak to match how you command the hand)
        self.action_space = spaces.Box(low=-1.0, high=1.0, shape=(action_dim,), dtype=np.float32)

        # Observation: will be the 10-dim vector in your proposal. Provide high/low bounds reasonably.
        high = np.ones(obs_dim, dtype=np.float32) * 1e3
        low = -high
        self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)

        # bookkeeping for smoothness
        self.prev_action = np.zeros(self.action_space.shape, dtype=np.float32)

        # placeholders for tracking targets (replace with actual target note sequence from MIDI)
        self.current_target = None
        self.time_since_step = 0.0

    '''def _build_obs(self):
        """
        Query self.env for needed low-level values and construct your 10-dim vector:
         - joint angles (2 per finger) (or a reduced summary)
         - joint velocities
         - key depression
         - key velocity
         - target note timing
         - current time step
         - finger force
        Replace environment accessors as needed.
        """
        # EXAMPLE placeholders:
        joint_angles = self.env.get_joint_angles()      # shape (10,) or whatever your env uses
        joint_vels = self.env.get_joint_velocities()
        key_depressions = self.env.get_key_depressions()  # per-key or aggregated
        key_vel = self.env.get_key_velocities()
        time_to_next = self._time_to_next_note()
        finger_force = self.env.get_finger_forces()

        # compress / stack / normalize to 10-dim (example only)
        obs = np.concatenate([
            np.ravel(joint_angles)[:2],
            np.ravel(joint_vels)[:2],
            np.array([key_depressions.mean(), key_vel.mean(), time_to_next, self.time_since_step]),
            np.array([finger_force.mean()])
        ]).astype(np.float32)

        # pad/crop to length
        if obs.shape[0] < self.observation_space.shape[0]:
            pad = np.zeros(self.observation_space.shape[0] - obs.shape[0], dtype=np.float32)
            obs = np.concatenate([obs, pad])
        else:
            obs = obs[:self.observation_space.shape[0]]
        return obs'''

    def _build_obs(self, obs_dict):
        """
        Build observation including fingering guidance
        """
        # Get current state (adapt keys to your environment)
        try:
            joint_positions = obs_dict.get("shadow_piano/hand/joint_positions", np.zeros(24))
            joint_vels = obs_dict.get("shadow_piano/hand/joint_velocities", np.zeros(24))
            key_pos = obs_dict.get("piano/key_positions", np.zeros(88))
            key_vel = obs_dict.get("piano/key_velocities", np.zeros(88))
        except:
            # Fallback if keys are different
            joint_positions = np.zeros(24)
            joint_vels = np.zeros(24)
            key_pos = np.zeros(88)
            key_vel = np.zeros(88)

        # Get target note info (including fingering!)
        if self.next_idx < len(self.target_notes):
            target = self.target_notes[self.next_idx]
            target_pitch = target['pitch']
            target_finger = target['finger']  # Which finger to use (1-5)
            time_to_note = target['time'] - self._env.physics.data.time
        else:
            target_pitch = 60  # Default middle C
            target_finger = 3  # Default middle finger
            time_to_note = 1.0

        # Normalize target pitch to [0, 1]
        normalized_pitch = (target_pitch - 21) / 88.0  # MIDI range 21-108

        # One-hot encode target finger
        finger_onehot = np.zeros(5)
        if 1 <= target_finger <= 5:
            finger_onehot[target_finger - 1] = 1.0

        # Compose observation (adjust to match obs_dim)
        features = np.concatenate([
            joint_positions[:2],              # 2: finger joint angles
            joint_vels[:2],                   # 2: finger joint velocities
            np.array([key_pos[target_pitch]]), # 1: target key position
            np.array([normalized_pitch]),      # 1: which key to press
            finger_onehot,                     # 5: which finger to use
            np.array([time_to_note]),          # 1: timing
        ])

        # Pad or truncate to obs_dim
        if features.shape[0] < self.obs_dim:
            features = np.pad(features, (0, self.obs_dim - features.shape[0]))
        else:
            features = features[:self.obs_dim]

        return features.astype(np.float32)

    def time_to_next_note(self):
      if self.next_idx >= len(self.target_notes):
        return 1.0
      target_t, pitch = self.target_notes[self.next_idx]
      # Fix: physics is a property, not a method - remove the ()
      return target_t - self._env.physics.data.time

    def step(self, action):
        # Map the normalized action [-1,1] to your env command (e.g., torque or position)
        env_action = self._map_action(action)
        self.env.step(env_action)
        self.time_since_step += self.dt

        obs = self._build_obs()

        # compute reward components
        rhit = self._compute_rhit()
        rtiming = self._compute_rtiming()
        renergy = - (np.linalg.norm(env_action) ** 2)
        rsmooth = - (np.linalg.norm(action - self.prev_action) ** 2)

        reward = (self.rw['w_hit'] * rhit +
                  self.rw['w_timing'] * rtiming +
                  self.rw['w_energy'] * renergy +
                  self.rw['w_smooth'] * rsmooth)

        done = False  # define terminal conditions (e.g., end of piece, timeout)
        info = {
            'rhit': rhit, 'rtiming': rtiming, 'renergy': renergy, 'rsmooth': rsmooth,
            'time': self.env.simulation_time()
        }

        self.prev_action = action.copy()
        return obs, reward, done, info

    def _map_action(self, action):
        # Convert normalized action to real actuation. Example: scale to torque/positions.
        # Replace with the real mapping your env expects.
        max_torque = 5.0
        return action * max_torque

    def _compute_rhit(self):
        # Check if target key depressed beyond threshold within a small window
        # This is placeholder logic — replace with actual key press detection
        key_dep = self.env.get_key_depressions_for_target(self.current_target)
        return 1.0 if np.any(key_dep > self.hit_threshold) else 0.0

    def _compute_rtiming(self):
        # Gaussian falloff around target note time
        if self.current_target is None:
            return 0.0
        dt = (self.env.simulation_time() - self.current_target['time'])
        return np.exp(- (dt ** 2) / (self.sigma_timing ** 2))

    def reset(self, seed=None, options=None):
        self.env.reset()
        self.prev_action = np.zeros_like(self.prev_action)
        self.time_since_step = 0.0
        # set the next target note from your schedule (MIDI)
        self.current_target = self._pop_next_target()
        return self._build_obs()

    def _pop_next_target(self):
        # pop the next note from scheduled list (implement according to how you load MIDI)
        return None

In [ ]:
import numpy as np
from dm_env import TimeStep, StepType

class CustomRoboWrapper:
    def __init__(self, env, target_notes, obs_dim=10, action_dim=5,
                 finger_only=True, w_hit=1.0, w_timing=1.0,
                 w_energy=0.01, w_smooth=0.01):
        self._env = env
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.finger_only = finger_only
        self.prev_action = np.zeros(self.action_dim)
        self.target_notes = target_notes  # Now with fingering info!
        self.next_idx = 0

        # Reward weights
        self.w_hit = w_hit
        self.w_timing = w_timing
        self.w_energy = w_energy
        self.w_smooth = w_smooth
        self.hit_threshold = 0.3  # Key depression threshold

        # Define which actuators control fingers (exclude wrist)
        # These indices will depend on your specific Shadow Hand model
        if self.finger_only:
            # Example: actuators 2-21 are finger joints (skip 0-1 which are wrist)
            self.finger_actuator_indices = list(range(2, 22))  # 20 finger actuators
        else:
            self.finger_actuator_indices = None

    def _map_action(self, action):
        """
        Map agent's action to environment actuators.
        If action_dim=5, each action controls one finger (simplified).
        If finger_only=True, only activate finger actuators.
        """
        full_dim = self._env.action_spec().shape[0]
        full_action = np.zeros(full_dim, dtype=np.float32)

        if self.finger_only and self.finger_actuator_indices:
            # Map to finger actuators only
            if self.action_dim == 5:
                # Simple model: each action controls one finger
                # Finger 1 (thumb) -> actuators 2-5
                # Finger 2 (index) -> actuators 6-9
                # Finger 3 (middle) -> actuators 10-13
                # Finger 4 (ring) -> actuators 14-17
                # Finger 5 (pinky) -> actuators 18-21

                for finger_idx in range(5):
                    # Each finger has ~4 joints
                    start_actuator = 2 + (finger_idx * 4)
                    end_actuator = start_actuator + 4

                    # Broadcast action to all joints of this finger
                    full_action[start_actuator:end_actuator] = action[finger_idx] * 2.0
            else:
                # Direct mapping: each action controls one actuator
                for i, actuator_idx in enumerate(self.finger_actuator_indices[:self.action_dim]):
                    full_action[actuator_idx] = action[i] * 2.0
        else:
            # Full control (including wrist)
            full_action[:self.action_dim] = action * 2.0

        return full_action

    '''def __init__(self, env, target_midi, obs_dim=45, action_dim=15):
        self._env = env
        self.obs_dim = obs_dim
        self.action_dim = action_dim
        self.prev_action = np.zeros(self.action_dim) # Initialize with custom action_dim
        self.target_notes = target_midi  # list of (time, pitch)
        self.next_idx = 0'''

    # ====  CUSTOM OBSERVATION =====
    def _build_obs(self, obs_dict):
      joint_positions = obs_dict["shadow_piano/hand/joint_positions"]
      joint_vels      = obs_dict["shadow_piano/hand/joint_velocities"]
      key_pos         = obs_dict["piano/key_positions"]
      key_vel         = obs_dict["piano/key_velocities"]

      # Compress to 10 dims (example)
      features = np.concatenate([
        joint_positions[:2],           # 2 dims: first 2 joint angles
        joint_vels[:2],                # 2 dims: first 2 joint velocities
        np.array([key_pos.mean()]),    # 1 dim: avg key depression
        np.array([key_vel.mean()]),    # 1 dim: avg key velocity
        np.array([self.time_to_next_note()]),  # 1 dim: timing
        np.array([self._env.physics.data.time]),  # 1 dim: current time
        np.zeros(2)                    # 2 dims: padding/future use
    ])

      return features[:self.obs_dim].astype(np.float32)

    # ==== CUSTOM ACTION MAPPING ====
    '''def _map_action(self, action):
        """
        Map a simple 3D or 5D continuous vector into the full actuator space.
        E.g. scale and broadcast torque to specific joints.
        """
        full_dim = self._env.action_spec().shape[0]
        full_action = np.zeros(full_dim, dtype=np.float32)

        # Example: control only 3 selected joints
        # The incoming action here should already be of shape (self.action_dim,)
        full_action[:self.action_dim] = action * 2.0
        return full_action'''

    # ==== TIMING UTILS ====
    def time_to_next_note(self):
        if self.next_idx >= len(self.target_notes):
            return 1.0
        target_t, pitch = self.target_notes[self.next_idx]
        return target_t - self._env.physics().data.time

    # ==== REWARD ====
    '''def compute_reward(self, ts, action):
        hit_reward = 0.0
        timing_reward = 0.0

        # Example: detect key press
        # keys = ts.observation["piano/key_positions"]
        # if keys[target_pitch] > threshold: hit_reward = 1.0

        # Gaussian timing reward
        dt = self.time_to_next_note()
        sigma = 0.05
        timing_reward = np.exp(- (dt**2) / (2 * sigma**2))

        # Energy & smoothness
        # Ensure 'action' here is the 3D action from the agent, not the full 45D env_action.
        energy = -0.01 * np.sum(action**2)
        smooth = -0.01 * np.sum((action - self.prev_action)**2)

        self.prev_action = action.copy() # Store the agent's 3D action

        return hit_reward + timing_reward + energy + smooth'''

    def compute_reward(self, ts, action):
        """
        Reward function with fingering guidance
        """
        r_hit = 0.0
        r_timing = 0.0
        r_finger = 0.0  # NEW: reward for using correct finger

        if self.next_idx < len(self.target_notes):
            target = self.target_notes[self.next_idx]
            target_time = target['time']
            target_pitch = target['pitch']
            target_finger = target['finger']

            try:
                key_positions = ts.observation.get("piano/key_positions", np.zeros(88))
            except:
                key_positions = np.zeros(88)

            # Check if target key is pressed
            if key_positions[target_pitch] > self.hit_threshold:
                r_hit = 1.0

                # Bonus for using correct finger
                # Check which finger applied the most force/movement
                active_finger = self._get_active_finger(action)
                if active_finger == target_finger:
                    r_finger = 0.5  # Bonus for correct fingering

                self.next_idx += 1  # Move to next note

            # Timing reward (Gaussian centered at target time)
            current_time = self._env.physics.data.time
            dt = current_time - target_time
            sigma_timing = 0.05
            r_timing = np.exp(-(dt**2) / (2 * sigma_timing**2))

        # Energy penalty (encourage efficiency)
        r_energy = -np.sum(action**2)

        # Smoothness penalty (discourage jerky movements)
        r_smooth = -np.sum((action - self.prev_action)**2)

        self.prev_action = action.copy()

        # Combined reward
        total_reward = (
            self.w_hit * r_hit +
            self.w_timing * r_timing +
            0.5 * r_finger +  # Fingering bonus
            self.w_energy * r_energy +
            self.w_smooth * r_smooth
        )

        return total_reward

    def _get_active_finger(self, action):
        """
        Determine which finger is most active based on action values
        """
        if self.action_dim == 5:
            # If action_dim=5, each dimension controls one finger
            most_active = np.argmax(np.abs(action)) + 1  # +1 because fingers are 1-indexed
            return most_active
        else:
            # More complex logic needed for higher action dims
            return 3  # Default to middle finger

    # ==== DM_ENV FORWARDING ====
    def reset(self):
        self.prev_action = np.zeros(self.action_dim)
        self.next_idx = 0
        ts = self._env.reset()
        obs = self._build_obs(ts.observation)
        return TimeStep(step_type=ts.step_type, reward=0.0, discount=1.0, observation=obs)

    def step(self, action):
        full_action = self._map_action(action)
        ts = self._env.step(full_action)
        obs = self._build_obs(ts.observation)
        rew = self.compute_reward(ts, action)
        return TimeStep(
            step_type=ts.step_type,
            reward=rew,
            discount=ts.discount,
            observation=obs
        )

    def observation_spec(self):
        from dm_env import specs
        return {
            "obs": specs.Array(shape=(self.obs_dim,), dtype=np.float32)
        }

    def action_spec(self):
        from dm_env import specs
        # This should reflect the action space *this wrapper* exposes to the agent,
        # not the underlying full environment's action space.
        # Assuming a continuous action space for torque/velocity commands, typically [-1, 1]
        return specs.BoundedArray(
            shape=(self.action_dim,),
            dtype=np.float32,
            minimum=-1.0,  # Assuming actions are normalized to [-1, 1]
            maximum=1.0,   # Assuming actions are normalized to [-1, 1]
            name="action"
        )

    def reward_spec(self):
        return self._env.reward_spec()

    def discount_spec(self):
        return self._env.discount_spec()

In [ ]:
def make_training_env():
    task = piano_with_shadow_hands.PianoWithShadowHands(
        midi=music.load("TwinkleTwinkleRousseau"),
        trim_silence=True,
        control_timestep=0.05,
        change_color_on_activation=True,
    )

    raw_env = composer_utils.Environment(
        task=task,
        strip_singleton_obs_buffer_dim=True,
        recompile_physics=False,
    )

    raw_env = CanonicalSpecWrapper(raw_env)

    #### CORRECT MIDI EXTRACTION ####
    midi = music.load("TwinkleTwinkleRousseau")

    # (time, pitch) pairs
    note_times = sorted(
        [(note.start_time, note.pitch) for note in midi.seq.notes],
        key=lambda x: x[0]
    )

    custom_dm_env = CustomRoboWrapper(raw_env, note_times)

    gym_env = DMEnvToGym(custom_dm_env)
    return gym_env

In [ ]:
# train_rl.py
from stable_baselines3 import PPO, SAC
from stable_baselines3.common.vec_env import DummyVecEnv

def make_training_env_with_fingerings(
    midi_source="TwinkleTwinkleRousseau",  # or path to MIDI/MusicXML
    obs_dim=12,      # Increased to include finger guidance
    action_dim=5,    # One action per finger
    finger_only=True,
    w_hit=2.0,
    w_timing=1.0,
    w_energy=0.005,
    w_smooth=0.01
):
    # Load task
    task = piano_with_shadow_hands.PianoWithShadowHands(
        midi=music.load(midi_source),
        trim_silence=True,
        control_timestep=0.05,
        change_color_on_activation=True,
    )

    raw_env = composer_utils.Environment(
        task=task,
        strip_singleton_obs_buffer_dim=True,
        recompile_physics=False,
    )
    raw_env = CanonicalSpecWrapper(raw_env)

    # Load MIDI with fingerings
    if midi_source == "TwinkleTwinkleRousseau":
        # Use manually created fingerings
        note_times = create_twinkle_twinkle_with_fingerings()
    else:
        # Try to load from file
        if midi_source.endswith('.xml'):
            note_times = load_musicxml_with_fingerings(midi_source)
        else:
            note_times = load_midi_with_fingerings(midi_source)

    # Wrap with custom observation/action/reward
    custom_dm_env = CustomRoboWrapper(
        raw_env,
        note_times,
        obs_dim=obs_dim,
        action_dim=action_dim,
        finger_only=finger_only,
        w_hit=w_hit,
        w_timing=w_timing,
        w_energy=w_energy,
        w_smooth=w_smooth
    )

    gym_env = DMEnvToGym(custom_dm_env)
    return gym_env

# Train
env = DummyVecEnv([lambda: make_training_env_with_fingerings()])

algo = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=128,
    n_epochs=10,
    verbose=1,
    tensorboard_log="./logs/ppo_fingerings",
    device='cpu'
)

algo.learn(total_timesteps=5_000_000)
algo.save("ppo_robopianist_fingerings")

'''
# Correctly pass a callable (a function that returns an environment instance) to DummyVecEnv
env = DummyVecEnv([lambda: make_training_env()])

algo = PPO(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    verbose=1,
    tensorboard_log="./logs/ppo",
    device='cpu'
)

algo.learn(1_000_000)
algo.save("ppo_robopianist")

algo = SAC(
    "MlpPolicy",
    env,
    learning_rate=3e-4,
    buffer_size=500_000,
    batch_size=256,
    verbose=1,
    tensorboard_log="./logs/sac"
)
algo.learn(1_000_000)
algo.save("sac_robopianist")
'''

AssertionError: low.shape doesn't match provided shape, low.shape: (), shape: (5,)

In [ ]:
def create_twinkle_twinkle_with_fingerings():
    """
    Manually specify (time, pitch, finger, duration) for Twinkle Twinkle
    Using C major scale: C=60, D=62, E=64, F=65, G=67, A=69
    """
    notes = [
        # "Twin-kle twin-kle lit-tle star"
        {'time': 0.0,  'pitch': 60, 'finger': 1, 'duration': 0.5, 'velocity': 100},  # C - Thumb
        {'time': 0.5,  'pitch': 60, 'finger': 1, 'duration': 0.5, 'velocity': 100},  # C - Thumb
        {'time': 1.0,  'pitch': 67, 'finger': 5, 'duration': 0.5, 'velocity': 100},  # G - Pinky
        {'time': 1.5,  'pitch': 67, 'finger': 5, 'duration': 0.5, 'velocity': 100},  # G - Pinky
        {'time': 2.0,  'pitch': 69, 'finger': 5, 'duration': 0.5, 'velocity': 100},  # A - Pinky
        {'time': 2.5,  'pitch': 69, 'finger': 5, 'duration': 0.5, 'velocity': 100},  # A - Pinky
        {'time': 3.0,  'pitch': 67, 'finger': 5, 'duration': 1.0, 'velocity': 100},  # G - Pinky (hold)

        # "How I won-der what you are"
        {'time': 4.0,  'pitch': 65, 'finger': 4, 'duration': 0.5, 'velocity': 100},  # F - Ring
        {'time': 4.5,  'pitch': 65, 'finger': 4, 'duration': 0.5, 'velocity': 100},  # F - Ring
        {'time': 5.0,  'pitch': 64, 'finger': 3, 'duration': 0.5, 'velocity': 100},  # E - Middle
        {'time': 5.5,  'pitch': 64, 'finger': 3, 'duration': 0.5, 'velocity': 100},  # E - Middle
        {'time': 6.0,  'pitch': 62, 'finger': 2, 'duration': 0.5, 'velocity': 100},  # D - Index
        {'time': 6.5,  'pitch': 62, 'finger': 2, 'duration': 0.5, 'velocity': 100},  # D - Index
        {'time': 7.0,  'pitch': 60, 'finger': 1, 'duration': 1.0, 'velocity': 100},  # C - Thumb (hold)
    ]

    return notes

import pandas as pd
import numpy as np

def load_fingerings_from_csv(csv_path, default_finger_assignment='optimal'):
    """
    Load note data from CSV and assign fingerings.
    """
    df = pd.read_csv(csv_path)

    has_fingerings = 'finger' in df.columns or 'fingering' in df.columns

    notes_with_fingerings = []

    for idx, row in df.iterrows():
        note_dict = {
            'time': float(row['t_on']),
            'pitch': int(row['pitch_number']),
            'duration': float(row['t_off'] - row['t_on']),
            'velocity': int(row['velocity']),
            'pitch_name': row['pitch_name']
        }

        if has_fingerings:
            finger_col = 'finger' if 'finger' in df.columns else 'fingering'
            note_dict['finger'] = int(row[finger_col])
        else:
            if default_finger_assignment == 'heuristic':
                note_dict['finger'] = assign_finger_by_pitch(row['pitch_number'])
            elif default_finger_assignment == 'round_robin':
                note_dict['finger'] = (idx % 5) + 1
            elif default_finger_assignment == 'optimal':
                if idx == 0:
                    note_dict['finger'] = assign_finger_by_pitch(row['pitch_number'])
                else:
                    prev_pitch = notes_with_fingerings[-1]['pitch']
                    prev_finger = notes_with_fingerings[-1]['finger']
                    note_dict['finger'] = assign_finger_optimally(
                        row['pitch_number'],
                        prev_pitch,
                        prev_finger
                    )

        notes_with_fingerings.append(note_dict)

    return notes_with_fingerings


def assign_finger_by_pitch(pitch):
    """Heuristic: assign fingers based on pitch range."""
    if pitch < 45:
        return 1  # Thumb
    elif pitch < 52:
        return 2  # Index
    elif pitch < 60:
        return 2  # Index
    elif pitch < 67:
        return 3  # Middle
    elif pitch < 74:
        return 4  # Ring
    else:
        return 5  # Pinky


def assign_finger_optimally(current_pitch, prev_pitch, prev_finger):
    """Assign finger to minimize hand movement."""
    interval = current_pitch - prev_pitch

    if interval == 0:
        return prev_finger

    if abs(interval) <= 4:
        if interval > 0:
            new_finger = min(prev_finger + 1, 5)
        else:
            new_finger = max(prev_finger - 1, 1)
        return new_finger
    else:
        return assign_finger_by_pitch(current_pitch)


def visualize_fingerings(notes_with_fingerings, max_notes=20):
    """Print a visual representation of fingerings."""
    print("=" * 80)
    print(f"FINGERING VISUALIZATION (first {max_notes} notes)")
    print("=" * 80)
    print(f"{'Time':>8} {'Pitch':>6} {'Note':>6} {'Finger':>8} {'Duration':>10}")
    print("-" * 80)

    for i, note in enumerate(notes_with_fingerings[:max_notes]):
        finger_viz = '█' * note['finger']
        print(f"{note['time']:8.3f} {note['pitch']:6d} {note.get('pitch_name', 'N/A'):>6} "
              f"{finger_viz:>8} {note['duration']:10.3f}")

    print("=" * 80)
    print(f"Total notes: {len(notes_with_fingerings)}")
    print("\nFinger usage distribution:")
    finger_counts = {i: 0 for i in range(1, 6)}
    for note in notes_with_fingerings:
        finger_counts[note['finger']] += 1

    for finger, count in finger_counts.items():
        percentage = 100 * count / len(notes_with_fingerings)
        bar = '█' * int(percentage / 2)
        print(f"  Finger {finger}: {bar} {count:3d} ({percentage:5.1f}%)")


def parse_csv_for_training(csv_path, strategy='optimal'):
    """
    Main function to parse CSV and prepare for training.
    """
    notes = load_fingerings_from_csv(csv_path, default_finger_assignment=strategy)
    notes = sorted(notes, key=lambda x: x['time'])
    visualize_fingerings(notes)
    return notes


def create_midi_from_notes(notes):
    """
    Create a MIDI object that RoboPianist can use from note list.
    """
    import pretty_midi

    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for note_dict in notes:
        note = pretty_midi.Note(
            velocity=note_dict.get('velocity', 100),
            pitch=note_dict['pitch'],
            start=note_dict['time'],
            end=note_dict['time'] + note_dict['duration']
        )
        instrument.notes.append(note)

    midi.instruments.append(instrument)

    # Return the PrettyMIDI object directly - RoboPianist can handle it
    return midi


def make_training_env_with_fingerings(
    midi_source="TwinkleTwinkleRousseau",
    csv_path=None,  # Add CSV path option
    fingering_strategy='optimal',
    obs_dim=17,
    action_dim=5,
    finger_only=True,
    w_hit=2.0,
    w_timing=1.0,
    w_energy=0.005,
    w_smooth=0.01
):
    """
    Create training environment with fingerings from CSV or built-in songs.
    """
    # Determine note source
    if csv_path is not None:
        # Load from CSV file
        print(f"Loading notes from CSV: {csv_path}")
        note_times = parse_csv_for_training(csv_path, strategy=fingering_strategy)
        midi_obj = create_midi_from_notes(note_times)
    elif midi_source == "TwinkleTwinkleRousseau":
        # Use manually created fingerings
        print("Using built-in Twinkle Twinkle with manual fingerings")
        note_times = create_twinkle_twinkle_with_fingerings()
        midi_obj = music.load("TwinkleTwinkleRousseau")
    else:
        # Load from RoboPianist's built-in library
        print(f"Loading built-in MIDI: {midi_source}")
        midi_obj = music.load(midi_source)

        # Extract notes from the MIDI
        midi_notes = midi_obj.seq.notes
        note_times = []
        for i, note in enumerate(sorted(midi_notes, key=lambda n: n.start_time)):
            if fingering_strategy == 'optimal' and i > 0:
                prev_pitch = note_times[-1]['pitch']
                prev_finger = note_times[-1]['finger']
                finger = assign_finger_optimally(note.pitch, prev_pitch, prev_finger)
            else:
                finger = assign_finger_by_pitch(note.pitch)

            note_times.append({
                'time': note.start_time,
                'pitch': note.pitch,
                'duration': note.end_time - note.start_time,
                'velocity': note.velocity,
                'finger': finger
            })

        print(f"Loaded {len(note_times)} notes")
        visualize_fingerings(note_times, max_notes=10)

    # Create task
    task = piano_with_shadow_hands.PianoWithShadowHands(
        midi=midi_obj,
        trim_silence=True,
        control_timestep=0.05,
        change_color_on_activation=True,
    )

    raw_env = composer_utils.Environment(
        task=task,
        strip_singleton_obs_buffer_dim=True,
        recompile_physics=False,
    )
    raw_env = CanonicalSpecWrapper(raw_env)

    custom_dm_env = CustomRoboWrapper(
        raw_env,
        note_times,
        obs_dim=obs_dim,
        action_dim=action_dim,
        finger_only=finger_only,
        w_hit=w_hit,
        w_timing=w_timing,
        w_energy=w_energy,
        w_smooth=w_smooth
    )

    gym_env = DMEnvToGym(custom_dm_env)
    return gym_env